In [12]:
import csv, json, os, numbers, pandas, re, scipy, scipy.sparse, shutil
import subprocess, sys, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [5]:
pandas.options.display.max_colwidth = 300

## Download ACS2015 File Templates for 5-year and 1-year data

In [101]:
src = 'https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/2015_1yr_Summary_FileTemplates.zip'
dest = 'capture/ACS2015_1year/2015_1yr_Summary_FileTemplates.zip'
download_file(src, dest)
templates = unzip_file(dest)

src = 'https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/2015_5yr_Summary_FileTemplates.zip'
dest = 'capture/ACS2015_5year/2015_5yr_Summary_FileTemplates.zip'
download_file(src, dest)
templates = unzip_file(dest)

capture/ACS2015_1year/2015_1yr_Summary_FileTemplates.zip already downloaded
capture/ACS2015_1year/2015_1yr_Summary_FileTemplates.zip already unzipped
Done, wrote 1397862 bytes to capture/ACS2015_5year/2015_5yr_Summary_FileTemplates.zip
Unzipping capture/ACS2015_5year/2015_5yr_Summary_FileTemplates.zip into capture/ACS2015_5year/2015_5yr_Summary_FileTemplates.tmp
Success, created capture/ACS2015_5year/2015_5yr_Summary_FileTemplates


In [6]:
!ls -l capture/ACS2015_1year/2015_1yr_Summary_FileTemplates/Templates | head

total 13192
-rw-rw-r-- 1 rsargent rsargent  25600 Aug  5  2016 2015_SFGeoFileTemplate.xls
-rw-rw-r-- 1 rsargent rsargent  92160 Aug  5  2016 Seq100.xls
-rw-rw-r-- 1 rsargent rsargent  60928 Aug  5  2016 Seq101.xls
-rw-rw-r-- 1 rsargent rsargent  88064 Aug  5  2016 Seq102.xls
-rw-rw-r-- 1 rsargent rsargent 115200 Aug  5  2016 Seq103.xls
-rw-rw-r-- 1 rsargent rsargent  99328 Aug  5  2016 Seq104.xls
-rw-rw-r-- 1 rsargent rsargent 111104 Aug  5  2016 Seq105.xls
-rw-rw-r-- 1 rsargent rsargent 107520 Aug  5  2016 Seq106.xls
-rw-rw-r-- 1 rsargent rsargent  88064 Aug  5  2016 Seq107.xls
ls: write error: Broken pipe


## Download ACS2015 5-year data (tract and block group)

In [33]:
!wget --header="User-Agent: Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.97 Safari/537.11" --header="Referer: http://xmodulo.com/" https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_entire_sf/Tracts_Block_Groups_Only.tar.gz

!mkdir -p capture/ACS2005_5year
!mv Tracts_Block_Groups_Only.tar.gz capture/ACS2005_5year

!cd capture/ACS2005_5year; tar xvfz Tracts_Block_Groups_Only.tar.gz >/dev/null

!wget --header="User-Agent: Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.97 Safari/537.11" https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_entire_sf/2015_ACS_Geography_Files.zip

!mv 2015_ACS_Geography_Files.zip capture/ACS2005_5year

unzip_file('capture/ACS2005_5year/2015_ACS_Geography_Files.zip')

--2017-07-22 10:59:54--  https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_entire_sf/Tracts_Block_Groups_Only.tar.gz
Resolving www2.census.gov (www2.census.gov)... 104.95.31.46, 2600:1408:7:2a5::208c, 2600:1408:7:291::208c
Connecting to www2.census.gov (www2.census.gov)|104.95.31.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3747109902 (3.5G) [application/x-gzip]
Saving to: ‘Tracts_Block_Groups_Only.tar.gz’

Tracts_Block_Groups 100%[===================>]   3.49G  4.91MB/s    in 14m 0s  

2017-07-22 11:13:55 (4.25 MB/s) - ‘Tracts_Block_Groups_Only.tar.gz’ saved [3747109902/3747109902]



## Read CSV utility functions

In [67]:
def read_acs2015_5year_template(seqno):
    path = 'capture/ACS2015_5year/2015_5yr_Summary_FileTemplates/2015_5yr_Templates/Seq%d.xls' % seqno
    if not os.path.exists(path):
        return None
    return pandas.read_excel(path)

# Combine template header and data into pandas frame
def read_acs2015_year_data(state, seqno):
    header = read_acs2015_5year_template(seqno)
    data = pandas.read_csv('capture/ACS2015_5year/group2/e20155%s%04d000.txt' % (state, seqno),
                            index_col=False,
                            dtype={'FILEID':numpy.str,
                                   'FILETYPE':numpy.str,
                                   'STUSAB':numpy.str,
                                   'CHARITER':numpy.str,
                                   'SEQUENCE':numpy.str,
                                   'LOGRECNO':numpy.str},
                            header=None,
                            names=header.columns.values)
    return data

In [25]:
dataset = 'acs2015_5year_tract2010'
column_dir = 'columncache'

## Write ACS2015 5-year description.html

In [89]:
description_path = column_dir + '/' + dataset + '/description.html'
force_regenerate = True

if os.path.exists(description_path) and not force_regenerate:
    print '{description_path} already exists, skipping'.format(**locals())
else:
    table_rows = []

    for seqno in range(1, 1000):
        template = read_acs2015_5year_template(seqno)
        if template is None:
            break
        for col in range(6, template.shape[1]):
            colname = template.columns.values[col]
            description = template.iloc[0,col]
            description = description.replace(':', '')
            description = re.sub(r'\s*%\s*', ' &mdash; ', description)
            table_rows.append(u'<tr><td>{dataset}.{colname}</td><td>{description}</td></tr>\n'.format(**locals()))

    html = '<table>' + ''.join(table_rows) + '</table>'

    open(description_path, 'w').write(html.encode('utf8'))
    print 'Wrote %d column names and descriptions to %s' % (len(table_rows), description_path)

Wrote 22767 column names and descriptions to columncache/acs2015_5year_tract2010/description.html


## Create ACS2015 block-level population

### Read 2010 block geoids and 2010 block populations

In [42]:
block_populations = numpy.load('columncache/census2010_block2010/p001001.numpy')
print 'block_populations has', sum(block_populations), 'total people'

block_populations has 308745538 total people


In [19]:
# block_geoids_2010 = [row[0] for row in query_psql("SELECT geoid2010 FROM sf1_2010_block_p001 order by blockidx2010")]
block_geoids_2010 = json.load(open('block_geoids_2010.json'))
print 'There are', len(block_geoids_2010), 'blocks'

assert(len(block_geoids_2010) + 1 == len(block_populations))

There are 11078297 blocks


### Compute 2010 population by tract and block indices from tract


In [43]:
tract_populations = {}
tract_block_indexes = {}

for block_index_minus_one, block_geoid in enumerate(block_geoids_2010):
    block_index = block_index_minus_one + 1
    tract_name = block_geoid[0:11]
    if tract_name not in tract_populations:
        tract_populations[tract_name] = 0
        tract_block_indexes[tract_name] = []
    tract_populations[tract_name] += block_populations[block_index]
    tract_block_indexes[tract_name].append(block_index)

print 'There are', len(tract_populations), 'tracts'
print 'tract_populations has', sum(tract_populations.values()), 'people'

There are 73057 tracts
tract_populations has 308745538 people


### Map tract identifiers to LOGRECNO using geography file

In [70]:
tract_to_logrecno = {}

def compute_tract_to_logrecno(state):
    geography = pandas.read_csv('capture/ACS2015_5year/2015_ACS_Geography_Files/g20155{state}.csv'.format(**locals()),
                                dtype=numpy.str,
                                index_col=False,
                                header=None,
                                keep_default_na=False,
                                na_values=[])

    nrows = geography.shape[0]
    print 'State {state} has {nrows} geography rows'.format(**locals())
    
    ntracts = 0
    tract_to_logrecno[state] = {}
    
    for r in range(0, geography.shape[0]):
        aggregation_level = geography.iloc[r, 2]
        if aggregation_level == '140': # census tract
            tract_identifier = geography.iloc[r, 48][7:]
            logrecno = geography.iloc[r, 4]
            tract_to_logrecno[state][tract_identifier] = logrecno
    
    print 'Found %d tracts for state %s' % (len(tract_to_logrecno[state]), state)

for state in state_names:
    compute_tract_to_logrecno(state)

State ak has 3634 geography rows
Found 167 tracts for state ak
State al has 8809 geography rows
Found 1181 tracts for state al
State ar has 8491 geography rows
Found 686 tracts for state ar
State az has 8495 geography rows
Found 1526 tracts for state az
State ca has 40451 geography rows
Found 8057 tracts for state ca
State co has 7725 geography rows
Found 1249 tracts for state co
State ct has 5398 geography rows
Found 833 tracts for state ct
State dc has 679 geography rows
Found 179 tracts for state dc
State de has 1327 geography rows
Found 218 tracts for state de
State fl has 20878 geography rows
Found 4245 tracts for state fl
State ga has 13032 geography rows
Found 1969 tracts for state ga
State hi has 2532 geography rows
Found 351 tracts for state hi
State ia has 11642 geography rows
Found 825 tracts for state ia
State id has 3048 geography rows
Found 298 tracts for state id
State il has 23877 geography rows
Found 3123 tracts for state il
State in has 12444 geography rows
Found 1511

### Interpolate and write columns for data file

In [91]:
# TODO: can we do this with a data frame then write out columns?

def interpolate_acs_file(state, seq):
    print 'Reading %s:%d' % (state, seq)
    data = read_acs2015_year_data(state, seq)

    print 'Mapping locrecno to row'
    logrecnos = data['LOGRECNO']

    logrecno_to_row = {}

    for r, logrecno in enumerate(logrecnos):
        logrecno_to_row[logrecno] = r
    
    col_names = data.columns.values[6:]
    print 'Iterating across %d columns' % len(col_names)
    for col_name in col_names:
        input_col = data[col_name]
        output_col_path = column_dir + '/' + dataset + '/' + col_name + '.float32'
        if os.path.exists(output_col_path):
            print '%s already exists, skipping' % output_col_path
            continue

        output_col = numpy.zeros(block_populations.size, dtype=numpy.float32)

        for tract in sorted(tract_to_logrecno[state].keys()):
            input_pop = input_col[logrecno_to_row[tract_to_logrecno[state][tract]]]
            if not isinstance(input_pop, numbers.Number):
                if input_pop == '.':
                    input_pop = 0
                else:
                    try:
                        input_pop = float(input_pop)
                    except:
                        print 'That population is'
                        print input_pop
                        print type(input_pop)
                        print '>%s<' % input_pop
                        input_pop = 0
            if not tract in tract_block_indexes:
                print 'missing tract {tract} from tract_block_indexes'.format(**locals())
            else:
                for block_index in tract_block_indexes[tract]:
                    if block_populations[block_index]:
                        output_col[block_index] = input_pop * float(block_populations[block_index]) / tract_populations[tract]
            
        output_col.tofile(output_col_path + '.tmp')
        os.rename(output_col_path + '.tmp', output_col_path)
        print 'Created %s' % output_col_path

for seq in range(97, 2000):
    interpolate_acs_file('pa', seq)

Reading pa:97
Mapping locrecno to row
Iterating across 245 columns
columncache/acs2015_5year_tract2010/B24125_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_004.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_005.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_006.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_007.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_008.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_009.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_010.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_011.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_012.float3

Mapping locrecno to row
Iterating across 245 columns
columncache/acs2015_5year_tract2010/B24125_246.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_247.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_248.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_249.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_250.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_251.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_252.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_253.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_254.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_255.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_256.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24125_257.float32 already exis

Mapping locrecno to row
Iterating across 245 columns
columncache/acs2015_5year_tract2010/B24126_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_004.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_005.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_006.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_007.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_008.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_009.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_010.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_011.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_012.float32 already exis

Mapping locrecno to row
Iterating across 245 columns
columncache/acs2015_5year_tract2010/B24126_246.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_247.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_248.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_249.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_250.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_251.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_252.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_253.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_254.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_255.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_256.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B24126_257.float32 already exis

Mapping locrecno to row
Iterating across 243 columns
columncache/acs2015_5year_tract2010/B25001_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25002_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25002_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25002_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003A_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003A_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003A_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003B_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B25003B_002.float32 already

...

Mapping locrecno to row
Iterating across 183 columns
columncache/acs2015_5year_tract2010/B27003_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_004.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_005.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_006.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_007.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_008.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_009.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_010.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_011.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27003_012.float32 already exis

Mapping locrecno to row
Iterating across 217 columns
columncache/acs2015_5year_tract2010/B27010_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_004.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_005.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_006.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_007.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_008.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_009.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_010.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_011.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27010_012.float32 already exis

Mapping locrecno to row
Iterating across 213 columns
columncache/acs2015_5year_tract2010/B27019_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_004.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_005.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_006.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_007.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_008.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_009.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_010.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_011.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B27019_012.float32 already exis

Mapping locrecno to row
Iterating across 75 columns
columncache/acs2015_5year_tract2010/B99011_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99011_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99011_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99012_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99012_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99012_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99021_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99021_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99021_003.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99031_001.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99031_002.float32 already exists, skipping
columncache/acs2015_5year_tract2010/B99031_003.float32 already exist

AttributeError: 'NoneType' object has no attribute 'columns'

In [ ]:
# TODO: can we do this with a data frame then write out columns?

def interpolate_acs_file(seq):
    output_cols = {}
    missing_tracts = {}
    for state in state_names:
        data = read_acs2015_year_data(state, seq)
    
        logrecnos = data['LOGRECNO']

        logrecno_to_row = {}

        col_names = data.columns.values[6:]
        print '%s:%d has %d columns' % (state, seq, len(col_names))
        assert len(col_names) < 500   # sanity check to avoid demanding too much RAM on hal15

        for r, logrecno in enumerate(logrecnos):
            logrecno_to_row[logrecno] = r
    
        for col_name in col_names:
            input_col = data[col_name]
                
            if not col_name in output_cols:
                output_cols[col_name] = numpy.zeros(block_populations.size, dtype=numpy.float32)
            output_col = output_cols[col_name]

            for tract in sorted(tract_to_logrecno[state].keys()):
                input_pop = input_col[logrecno_to_row[tract_to_logrecno[state][tract]]]
                if not isinstance(input_pop, numbers.Number):
                    if input_pop == '.':
                        input_pop = 0
                    else:
                        try:
                            input_pop = float(input_pop)
                        except:
                            print 'That population is'
                            print input_pop
                            print type(input_pop)
                            print '>%s<' % input_pop
                            input_pop = 0
                            
                            
                if not tract in tract_block_indexes:
                    missing_tracts[tract] = True
                else:
                    for block_index in tract_block_indexes[tract]:
                        if block_populations[block_index]:
                            output_col[block_index] = input_pop * float(block_populations[block_index]) / tract_populations[tract]
            
    print 'Missing tracts: %s' % (sorted(missing_tracts.keys()))

    for col_name in sorted(output_cols.keys()):
        output_col_path = column_dir + '/' + dataset + '/' + col_name + '.float32'
        output_cols[col_name].tofile(output_col_path + '.tmp')
        os.rename(output_col_path + '.tmp', output_col_path)
        print 'Created %s with sum %f' % (output_col_path, output_cols[col_name].sum())
    
        
for seq in range(1, 1000):
    interpolate_acs_file(seq)

ak:1 has 2 columns
al:1 has 2 columns
ar:1 has 2 columns
az:1 has 2 columns
ca:1 has 2 columns
co:1 has 2 columns
ct:1 has 2 columns
dc:1 has 2 columns
de:1 has 2 columns
fl:1 has 2 columns
ga:1 has 2 columns
hi:1 has 2 columns
ia:1 has 2 columns
id:1 has 2 columns
il:1 has 2 columns
in:1 has 2 columns
ks:1 has 2 columns
ky:1 has 2 columns
la:1 has 2 columns
ma:1 has 2 columns
md:1 has 2 columns
me:1 has 2 columns
mi:1 has 2 columns
mn:1 has 2 columns
mo:1 has 2 columns
ms:1 has 2 columns
mt:1 has 2 columns
nc:1 has 2 columns
nd:1 has 2 columns
ne:1 has 2 columns
nh:1 has 2 columns
nj:1 has 2 columns
nm:1 has 2 columns
nv:1 has 2 columns
ny:1 has 2 columns
oh:1 has 2 columns
ok:1 has 2 columns
or:1 has 2 columns
pa:1 has 2 columns
ri:1 has 2 columns
sc:1 has 2 columns
sd:1 has 2 columns
tn:1 has 2 columns
tx:1 has 2 columns
ut:1 has 2 columns
va:1 has 2 columns
vt:1 has 2 columns
wa:1 has 2 columns
wi:1 has 2 columns
wv:1 has 2 columns
wy:1 has 2 columns
Missing tracts: ['02158000100',

Created columncache/acs2015_5year_tract2010/B01001C_001.float32 with sum 2547382.000000
Created columncache/acs2015_5year_tract2010/B01001C_002.float32 with sum 1262749.500000
Created columncache/acs2015_5year_tract2010/B01001C_003.float32 with sum 94353.976562
Created columncache/acs2015_5year_tract2010/B01001C_004.float32 with sum 100833.007812
Created columncache/acs2015_5year_tract2010/B01001C_005.float32 with sum 103020.000000
Created columncache/acs2015_5year_tract2010/B01001C_006.float32 with sum 64914.992188
Created columncache/acs2015_5year_tract2010/B01001C_007.float32 with sum 44572.996094
Created columncache/acs2015_5year_tract2010/B01001C_008.float32 with sum 109829.007812
Created columncache/acs2015_5year_tract2010/B01001C_009.float32 with sum 94723.992188
Created columncache/acs2015_5year_tract2010/B01001C_010.float32 with sum 87288.992188
Created columncache/acs2015_5year_tract2010/B01001C_011.float32 with sum 163969.968750
Created columncache/acs2015_5year_tract2010/B0

Created columncache/acs2015_5year_tract2010/B01001F_006.float32 with sum 377384.000000
Created columncache/acs2015_5year_tract2010/B01001F_007.float32 with sum 273092.062500
Created columncache/acs2015_5year_tract2010/B01001F_008.float32 with sum 721748.000000
Created columncache/acs2015_5year_tract2010/B01001F_009.float32 with sum 716990.750000
Created columncache/acs2015_5year_tract2010/B01001F_010.float32 with sum 707362.062500
Created columncache/acs2015_5year_tract2010/B01001F_011.float32 with sum 1214157.000000
Created columncache/acs2015_5year_tract2010/B01001F_012.float32 with sum 888791.187500
Created columncache/acs2015_5year_tract2010/B01001F_013.float32 with sum 490141.156250
Created columncache/acs2015_5year_tract2010/B01001F_014.float32 with sum 200893.000000
Created columncache/acs2015_5year_tract2010/B01001F_015.float32 with sum 77188.015625
Created columncache/acs2015_5year_tract2010/B01001F_016.float32 with sum 23101.000000
Created columncache/acs2015_5year_tract2010/

/home/rsargent/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (99,100,101) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


nc:3 has 124 columns
nd:3 has 124 columns
ne:3 has 124 columns
nh:3 has 124 columns
nj:3 has 124 columns
nm:3 has 124 columns
nv:3 has 124 columns
ny:3 has 124 columns
oh:3 has 124 columns
ok:3 has 124 columns
or:3 has 124 columns
pa:3 has 124 columns
ri:3 has 124 columns


...

Created columncache/acs2015_5year_tract2010/B08128_031.float32 with sum 7361654.000000
Created columncache/acs2015_5year_tract2010/B08128_032.float32 with sum 5364440.500000
Created columncache/acs2015_5year_tract2010/B08128_033.float32 with sum 5270533.500000
Created columncache/acs2015_5year_tract2010/B08128_034.float32 with sum 93906.007812
Created columncache/acs2015_5year_tract2010/B08128_035.float32 with sum 744049.812500
Created columncache/acs2015_5year_tract2010/B08128_036.float32 with sum 407376.968750
Created columncache/acs2015_5year_tract2010/B08128_037.float32 with sum 289014.031250
Created columncache/acs2015_5year_tract2010/B08128_038.float32 with sum 294593.031250
Created columncache/acs2015_5year_tract2010/B08128_039.float32 with sum 256656.984375
Created columncache/acs2015_5year_tract2010/B08128_040.float32 with sum 5521.000000
Created columncache/acs2015_5year_tract2010/B08128_041.float32 with sum 3996067.500000
Created columncache/acs2015_5year_tract2010/B08128_04

/home/rsargent/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (96,97,98,99,100,101,102,103,104,105,106,107,108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


al:27 has 235 columns
ar:27 has 235 columns
az:27 has 235 columns
ca:27 has 235 columns
co:27 has 235 columns
ct:27 has 235 columns
dc:27 has 235 columns
de:27 has 235 columns
fl:27 has 235 columns
ga:27 has 235 columns
hi:27 has 235 columns
ia:27 has 235 columns
id:27 has 235 columns
il:27 has 235 columns
in:27 has 235 columns
ks:27 has 235 columns
ky:27 has 235 columns
la:27 has 235 columns
ma:27 has 235 columns
md:27 has 235 columns
me:27 has 235 columns
mi:27 has 235 columns
mn:27 has 235 columns
mt:38 has 245 columns
nc:38 has 245 columns
nd:38 has 245 columns
ne:38 has 245 columns
nh:38 has 245 columns
nj:38 has 245 columns
nm:38 has 245 columns
nv:38 has 245 columns
ny:38 has 245 columns
oh:38 has 245 columns
ok:38 has 245 columns
or:38 has 245 columns
pa:38 has 245 columns
ri:38 has 245 columns
sc:38 has 245 columns
sd:38 has 245 columns
tn:38 has 245 columns
tx:38 has 245 columns
ut:38 has 245 columns
va:38 has 245 columns
vt:38 has 245 columns
wa:38 has 245 columns
wi:38 has 

Created columncache/acs2015_5year_tract2010/B12002_020.float32 with sum 4127.000000
Created columncache/acs2015_5year_tract2010/B12002_021.float32 with sum 22298.000000
Created columncache/acs2015_5year_tract2010/B12002_022.float32 with sum 685530.812500
Created columncache/acs2015_5year_tract2010/B12002_023.float32 with sum 2711141.500000
Created columncache/acs2015_5year_tract2010/B12002_024.float32 with sum 4790904.000000
Created columncache/acs2015_5year_tract2010/B12002_025.float32 with sum 5524853.500000
Created columncache/acs2015_5year_tract2010/B12002_026.float32 with sum 6165478.500000
Created columncache/acs2015_5year_tract2010/B12002_027.float32 with sum 6376196.000000
Created columncache/acs2015_5year_tract2010/B12002_028.float32 with sum 6755629.500000
Created columncache/acs2015_5year_tract2010/B12002_029.float32 with sum 6489671.500000
Created columncache/acs2015_5year_tract2010/B12002_030.float32 with sum 5924252.000000
Created columncache/acs2015_5year_tract2010/B1200

Created columncache/acs2015_5year_tract2010/B12002_117.float32 with sum 5399360.000000
Created columncache/acs2015_5year_tract2010/B12002_118.float32 with sum 5793722.500000
Created columncache/acs2015_5year_tract2010/B12002_119.float32 with sum 6248045.000000
Created columncache/acs2015_5year_tract2010/B12002_120.float32 with sum 6427832.000000
Created columncache/acs2015_5year_tract2010/B12002_121.float32 with sum 6777558.500000
Created columncache/acs2015_5year_tract2010/B12002_122.float32 with sum 6385803.500000
Created columncache/acs2015_5year_tract2010/B12002_123.float32 with sum 5552130.500000
Created columncache/acs2015_5year_tract2010/B12002_124.float32 with sum 6997396.500000
Created columncache/acs2015_5year_tract2010/B12002_125.float32 with sum 2694492.000000
Created columncache/acs2015_5year_tract2010/B12002_126.float32 with sum 477730.906250
Created columncache/acs2015_5year_tract2010/B12002_127.float32 with sum 6468974.500000
Created columncache/acs2015_5year_tract2010/

In [80]:
len(tract_block_indexes.keys())

73057

In [57]:
!ls -l columncache/acs2015_5year_tract2010/B08006_002.float32

-rw-rw-r-- 1 rsargent rsargent 44313192 Sep  1 18:34 columncache/acs2015_5year_tract2010/B08006_002.float32


In [59]:
x=numpy.memmap('columncache/acs2015_5year_tract2010/B08006_002.float32', dtype=numpy.float32, mode='r')

In [61]:
x.sum()

5011539.5